In [34]:
#set up dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [27]:
#create list of 500+ random cities

#create random coordinates
lat_list = np.random.uniform(-90.00, 90.00, 1500)
lng_list = np.random.uniform(-180.00, 180.00, 1500)

#combine list to create coodinates
coords_list = list(zip(lat_list, lng_list))

In [32]:
#set up lists for finding cities using citipy
cities = []
countries = []

for coords in coords_list:
    
    city = citipy.nearest_city(coords[0], coords[1]).city_name
    country = citipy.nearest_city(coords[0], coords[1]).country_code
    
    if city not in cities:
        cities.append(city)

len(cities)

599

In [57]:
#perform API calls through OpenWeatherMap API

#create function to extract data from API:
def extract_data(data):
    return {
        "name" : data["name"],
        "lat" : data["coord"]["lat"],
        "lng" : data["coord"]["lon"],
        "max_temp_f" : data["main"]["temp_max"],
        "humidity" : data["main"]["humidity"],
        "cloudiness" : data["clouds"]["all"],
        "windspeed" : data["wind"]["speed"],
        "country" : data["sys"]["country"],
        "date" : data["dt"]
    }